In [37]:
import pandas as pd
from tqdm import tqdm_notebook as tnote

In [2]:
df = pd.read_csv('ArticlesApril2018.csv')

In [3]:
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [4]:
df.columns

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')

In [7]:
df.headline.isnull().value_counts()

False    1324
Name: headline, dtype: int64

In [9]:
headline = []
headline.extend(list(df.headline.values))

In [10]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [13]:
idx = df[df.headline=='Unknown'].index

In [15]:
df.drop(index = idx,inplace = True)

In [16]:
headline = []
headline.extend(list(df.headline.values))

In [20]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [26]:
from string import punctuation

In [31]:
def repreprocessing(s):
    s = s.encode('utf8').decode('ascii', 'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

In [32]:
text = [repreprocessing(x) for x in headline]
text [:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [33]:
from keras_preprocessing.text import Tokenizer

In [34]:
t = Tokenizer()

In [35]:
t.fit_on_texts(text)
vocab_size = len(t.word_index)+1
vocab_size

3494

In [39]:
sequences = list()

for line in tnote(text):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [40]:
index_to_word={}
for key, value in t.word_index.items():
    index_to_word[value] = key
index_to_word[582]

'offer'

In [41]:
max_len = max(len(i) for i in sequences)
print(max_len)

24


In [42]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [44]:
sequences = pad_sequences( sequences, maxlen = max_len, padding= 'pre')
sequences[:3]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          99,  269],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   99,
         269,  371],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   99,  269,
         371, 1115]])

In [46]:
import numpy as np
sequences = np.array(sequences)

In [47]:
X = sequences[:, :-1]
y = sequences[:, -1]

In [48]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [49]:
from keras.utils import to_categorical

In [51]:
y = to_categorical(y, num_classes = vocab_size)

In [52]:
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential

In [72]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y,epochs=10, verbose=2)

Epoch 1/10
 - 8s - loss: 7.6379 - acc: 0.0297
Epoch 2/10
 - 6s - loss: 7.1124 - acc: 0.0278
Epoch 3/10
 - 7s - loss: 6.9772 - acc: 0.0381
Epoch 4/10
 - 7s - loss: 6.8443 - acc: 0.0406
Epoch 5/10
 - 7s - loss: 6.6891 - acc: 0.0443
Epoch 6/10
 - 7s - loss: 6.5120 - acc: 0.0477
Epoch 7/10
 - 7s - loss: 6.3202 - acc: 0.0527
Epoch 8/10
 - 7s - loss: 6.1265 - acc: 0.0546
Epoch 9/10
 - 7s - loss: 5.9326 - acc: 0.0600
Epoch 10/10
 - 7s - loss: 5.7464 - acc: 0.0639


In [59]:
max_len

24

In [74]:
def sentencs_generation(model, t, current_word, n):
    init_word = current_word
    sentence  = ''
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=23, padding='pre')
        result = model.predict_classes(encoded, verbose=0)
        for word, index in t.word_index.items():
            if index == result:
                break
        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word
    sentence = init_word + sentence
    return sentence

In [75]:
print(sentencs_generation(model, t, 'how', 10))

how the new place of the new glass of a camera


In [76]:
print(sentencs_generation(model, t, 'i', 10))

i the place of a camera war a new glass of
